# 1º) CONEXION SQL PARA BBDD PARADAS DE BICIS

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests

sqlalchemy.__version__

'1.4.32'

In [2]:
#connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
#connection_string

In [3]:
#engine = create_engine(connection_string)

In [4]:
#inspector = inspect(engine)
#inspector.get_table_names()

In [5]:
#df = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
#df

In [6]:
tst_url= '../datasets/bicimad_stations.csv'

In [7]:
df = pd.read_csv(tst_url, sep=',')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    264 non-null    int64 
 1   name                  264 non-null    object
 2   light                 264 non-null    int64 
 3   number                264 non-null    object
 4   address               264 non-null    object
 5   activate              264 non-null    int64 
 6   no_available          264 non-null    int64 
 7   total_bases           264 non-null    int64 
 8   dock_bikes            264 non-null    int64 
 9   free_bases            264 non-null    int64 
 10  reservations_count    264 non-null    int64 
 11  geometry.type         264 non-null    object
 12  geometry.coordinates  264 non-null    object
dtypes: int64(8), object(5)
memory usage: 26.9+ KB


In [9]:
#df con la información que necesitamos

df_bic = df[['name', 'address', 'geometry.coordinates']]
df_bic

,name,address,geometry.coordinates
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]"
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]"
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]"
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]"
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]"
...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]"
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]"
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]"
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]"


In [10]:
#df_bic[['location.longitude','location.latitude']]=df_bic.geometry.coordinates.str.split(',',expand=True)
#df_bic

In [11]:
df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)
df_bic

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10372\3923452401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10372\3923452401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)


,name,address,geometry.coordinates,location.longitude.bic,location.latitude.bic
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]",[-3.7018341,40.4172137]
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]",[-3.701602938060457,40.41731271011562]
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]",[-3.7058415,40.4205886]
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]",[-3.7069171,40.4302937]
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]",[-3.7025875,40.4285524]
...,...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]",[-3.72997,40.43896]
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]",[-3.72699,40.44375]
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]",[-3.72693,40.44342]
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]",[-3.7272945,40.4483322]


In [12]:
df_bic['location.longitude.bic'] 
df_bic['location.longitude.bic'] = df_bic['location.longitude.bic'].apply(lambda x: float(x[1:]))
df_bic['location.latitude.bic'] = df_bic['location.latitude.bic'].apply(lambda x: float(x[:-1]))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10372\3728200556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bic['location.longitude.bic'] = df_bic['location.longitude.bic'].apply(lambda x: float(x[1:]))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10372\3728200556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bic['location.latitude.bic'] = df_bic['location.latitude.bic'].apply(lambda x: float(x[:-1]))


### DF DEFINITIVO PARADAS DE BICIS = df_bic

In [13]:
df_bic = df_bic[['name', 'address', 'location.latitude.bic', 'location.longitude.bic']]
df_bic

,name,address,location.latitude.bic,location.longitude.bic
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


# 2º) CONEXIÓN API MONUMENTOS DE MADRID

In [14]:
endpoint = 'https://datos.madrid.es/egob'
url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
response = requests.get(endpoint + url)
response
print(type(response))

<class 'requests.models.Response'>


In [15]:
status = response.status_code
status

200

In [16]:
json_data = response.json()
print(type(json_data))
print(len(json_data))
json_data.keys()

<class 'dict'>
2


dict_keys(['@context', '@graph'])

In [17]:
df = pd.DataFrame(pd.json_normalize(json_data['@graph']))
df.head(5)

,@id,id,title,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige...",A las víctimas del Holocausto,NaN
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,A los Abuelos,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,408781,A Neruda-El Ser alado,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...,A Neruda-El Ser alado,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,408836,Abogados Laboristas,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...,Abogados Laboristas,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,408920,Abstracta I,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de...",Abstracta I,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [18]:
#ojo que hay nulos en localizaciones que será clave para obtener luego la más cercana.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   @id                             1848 non-null   object 
 1   id                              1848 non-null   object 
 2   title                           1848 non-null   object 
 3   relation                        1848 non-null   object 
 4   references                      1848 non-null   object 
 5   address.district.@id            1845 non-null   object 
 6   address.locality                1845 non-null   object 
 7   address.postal-code             1848 non-null   object 
 8   address.street-address          1848 non-null   object 
 9   location.latitude               1792 non-null   float64
 10  location.longitude              1792 non-null   float64
 11  organization.organization-desc  1848 non-null   object 
 12  organization.organization-name  18

In [19]:
#Revisando los valores nulos de la tabla

null_cols = df.isnull().sum()
null_cols[null_cols > 0]

address.district.@id      3
address.locality          3
location.latitude        56
location.longitude       56
address.area.@id        137
dtype: int64

### DF DEFINITIVO MONUMENTOS MADRID = df_monu

In [20]:
# vamos a crear un df con las columnas e información que nos interesan eliminando todos aquellos destinos que no tengan
# coordenadas.

df_monu = df[['title', 'address.street-address', 'location.latitude' , 'location.longitude']]
df_monu = df_monu.dropna()

In [21]:
df_monu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 1847
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   1792 non-null   object 
 1   address.street-address  1792 non-null   object 
 2   location.latitude       1792 non-null   float64
 3   location.longitude      1792 non-null   float64
dtypes: float64(2), object(2)
memory usage: 70.0+ KB


# 3º) DISTANCIA MENOR ENTRE PUNTOS

### Funciones facilitadas

In [22]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

In [23]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

### Test de las funciones

In [24]:
to_mercator(df_bic['location.latitude.bic'][0], df_bic['location.longitude.bic'][0])

0    POINT (4499223.648 -412373.285)
dtype: geometry

In [25]:
to_mercator(df_monu['location.latitude'][0], df_monu['location.longitude'][0])

0    POINT (4504467.691 -401410.940)
dtype: geometry

In [26]:
distance_meters(df_monu['location.latitude'][0], df_monu['location.longitude'][0], df_bic['location.latitude.bic'][0], df_bic['location.longitude.bic'][0])

0    12152.077352
dtype: float64

### Magia

In [35]:
%%time

df_nuevo = df_monu.head(1)
df_test = pd.concat([df_nuevo, df_bic], axis=1).ffill()
df_test['Distancia'] = df_test.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], x['location.latitude.bic'], x['location.longitude.bic']), axis=1)
zz = df_test.sort_values(by = 'Distancia')


CPU times: total: 5.33 s
Wall time: 5.33 s


In [36]:
zz.head(5)

,title,address.street-address,location.latitude,location.longitude,key,name,address,location.latitude.bic,location.longitude.bic,key,Distancia
232,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,0.0,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,40.443705,-3.648958,0,5559.887276
217,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,0.0,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77,40.429706,-3.640994,0,5682.027353
231,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,0.0,229 - C.D. Concepción,Calle Jose del Hierro nº 5,40.437151,-3.648359,0,5841.282012
234,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,0.0,232 - José María Pereda,Calle José María de Pereda nº 1,40.432721,-3.648531,0,6126.918816
233,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,0.0,231 - Derechos Humanos,Calle Derechos Humanos nº 12,40.439657,-3.655766,0,6438.264535


In [37]:
%%time

df_nuevo = df_monu.head(2)
df_test = pd.concat([df_nuevo, df_bic], axis=1).ffill()
df_test['Distancia'] = df_test.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], x['location.latitude.bic'], x['location.longitude.bic']), axis=1)
zz2 = df_test.sort_values(by = 'Distancia')

CPU times: total: 5.36 s
Wall time: 5.35 s


In [30]:
zz2.head()

,title,address.street-address,location.latitude,location.longitude,name,address,location.latitude.bic,location.longitude.bic,Distancia
251,A los Abuelos,C Alicún,40.479812,-3.661719,249 - Manuel Caldeiro,Paseo de la Castellana nº 298,40.479380,-3.685320,2633.107767
254,A los Abuelos,C Alicún,40.479812,-3.661719,252 - Chamartín,Calle Rodríguez Jaén nº 40,40.471600,-3.683910,2638.783266
256,A los Abuelos,C Alicún,40.479812,-3.661719,254 - Pío XII,Avenida Pío XII nº 3,40.460410,-3.677120,2759.738702
219,A los Abuelos,C Alicún,40.479812,-3.661719,217 - Pedro Rico,Calle Pedro Rico nº 4,40.481086,-3.688463,2986.641380
250,A los Abuelos,C Alicún,40.479812,-3.661719,248 - Torre Cepsa,Calle Sinesio Delgado nº 2,40.474350,-3.687970,2990.733573


In [38]:
df_bic['key'] = 0
df_monu['key'] = 0

In [39]:
df_pre = df_bic.merge(df_monu, on='key', how='left')#, lsuffix= '_1', rsuffix= '_2')
df_pre

,name,address,location.latitude.bic,location.longitude.bic,key,title,address.street-address,location.latitude,location.longitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558
1,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A los Abuelos,C Alicún,40.479812,-3.661719
2,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419
3,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abogados Laboristas,C Atocha 55,40.412911,-3.700017
4,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abstracta I,,40.421746,-3.662293
...,...,...,...,...,...,...,...,...,...
473083,261 - Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,0,Álvaro Iglesias Sánchez,CALLE CARRANZA 7,40.429000,-3.703131
473084,261 - Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,0,Ángeles de la Paz,Avda Bruselas,40.443023,-3.664153
473085,261 - Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,0,Árbol de la Vida,JARDIN TRES CULTURAS,40.464322,-3.603558
473086,261 - Facultad Derecho,Avenida Complutense nº 23,40.451090,-3.729370,0,Íñigo Arista,Pza Oriente,40.417710,-3.712172


In [40]:
%%time

#df_pre['Distancia'] = df_pre.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], 
                                                             x['location.latitude.bic'], x['location.longitude.bic']), axis=1)

CPU times: total: 2h 47min 37s
Wall time: 2h 47min 39s


In [42]:
#data = df_pre.to_csv('../datasets/df.csv', index=False)

### Ya tenemos el DF_pre guardado

In [49]:
tst_url2 = '../datasets/df.csv'

In [50]:
df_def = pd.read_csv(tst_url2, sep=',')

In [51]:
df_def.head()

,name,address,location.latitude.bic,location.longitude.bic,key,title,address.street-address,location.latitude,location.longitude,Distancia
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,12152.077352
1,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A los Abuelos,C Alicún,40.479812,-3.661719,8281.468285
2,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,6773.299650
3,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,520.142426
4,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abstracta I,,40.421746,-3.662293,4439.554041


In [44]:
#df_pre.groupby(['address.street-address']).agg('min')

In [48]:
zz5 = df_pre.sort_values(by = ['title', 'Distancia'])
zz5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473088 entries, 410370 to 401407
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   name                    473088 non-null  object 
 1   address                 473088 non-null  object 
 2   location.latitude.bic   473088 non-null  float64
 3   location.longitude.bic  473088 non-null  float64
 4   key                     473088 non-null  int64  
 5   title                   473088 non-null  object 
 6   address.street-address  473088 non-null  object 
 7   location.latitude       473088 non-null  float64
 8   location.longitude      473088 non-null  float64
 9   Distancia               473088 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 39.7+ MB
